In [2]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np

env = gym.make('CartPole-v1')

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
state = np.array(env.reset()[0])

state_tensor = tf.convert_to_tensor(state)
state_tensor = tf.expand_dims(state_tensor, 0)

state_tensor

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[ 0.00523372,  0.02015132,  0.00278229, -0.04800152]],
      dtype=float32)>

In [5]:
#create linear layer

class Linear(layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super().__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer='random_normal', trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer='zeros', trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [ ]:
#create model with custom layer

class MLP(keras.layers.Layer):

    def __init__(self):
        super().__init__()
        self.observations = Linear(4)
        self.hidden = Linear(32)
        self.action = Linear(2)

    def call(self, inputs):
        x = self.observations(inputs)
        x = tf.nn.relu(x)
        x = self.hidden(x)
        x = tf.nn.relu(x)
        return self.action(x)


mlp = MLP()

n_observations = 4
n_actions = 2
n_hidden = 128
batch_size = 32


def create_q_model():
    observations = layers.Input(shape=(n_observations,))
    hidden = layers.Dense(n_hidden, activation='relu')(observations)
    action = layers.Dense(n_actions, activation='softmax')(hidden)
    
    return keras.Model(inputs=observations, outputs=action)

policy_network = create_q_model()
target_network = create_q_model()

In [7]:
action_history = []
rewards_history = []
state_history = []
state_next_history = []
done_history = []

In [8]:
epsilon = 1.0

for timestep in range(10000):

    if epsilon > np.random.rand(1)[0]:
        action = np.random.choice(2)
    else:
        state_tensor = tf.convert_to_tensor(state)
        state_tensor = tf.expand_dims(state_tensor, 0)
        action_probs = policy_network(state_tensor, training=False)
        action = tf.argmax(action_probs[0]).numpy()

    epsilon = epsilon*0.99
    
    state_next, reward, done, _, _= env.step(action)
    
    action_history.append(action)
    rewards_history.append(reward)
    state_history.append(state)
    state_next_history.append(state_next)
    done_history.append(done)
        
    state_next = np.array(state_next)
    state = state_next

NameError: name 'policy_network' is not defined